# Collection of Twitter data for the <font color='red'>#Day-zero</font> sentiment ananlysis
Data is scraped from twitter relating to the waer  crisis issue in Cape Town

- drought sort of began in 2015 in the Western cape
- Cape town Details:
    - -33.9249° S, 18.4241° E
- Countries we do business in:


### Loading Packages

In [17]:
import numpy as np
import pandas as pd
import sqlite3
import json

### Authotrization

After perusing a few tweets as well as news articles, the following keywords have been suggested

In [18]:
conn = sqlite3.connect('test.db')
c = conn.cursor()
table = 'tweets'
for row in c.execute('SELECT text FROM ' + table + ' LIMIT 3'):
        print(row)

("b'Conspiracy https://t.co/rLZUteTM4S'",)
("b'Meaning farms of cape town are more important than human beings. This shocked\\xf0\\x9f\\x98\\xa8\\xf0\\x9f\\x98\\xa8 https://t.co/cpzzfFiRzE'",)
("b'#DayZero \\xe2\\x80\\x93 the day Cape Town is expected to turn off fresh water supply to suburbs \\xe2\\x80\\x93 is pegged at April 12 https://t.co/8LkSkvAQh5'",)
